In [2]:
%matplotlib inline
import numpy as np
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
import datetime as dt
import pandas as pd
from StringIO import StringIO

In [3]:
import re

def extractURLsatellite(fileURL, satName):
    """
    Function to extract the URLs for a specific satellite from the IMOS URLs list
    
    Parameters:
    -----------
    
    - fileURL : IMOS URLs list as a txt file
    - satName : name of the satellite such as JASON-2 JASON-3 
    
    Ouputs:
    ------
    
    - getFiles : list of URLs for the desired satellite
    
    """
    
    getFiles = []
    
    with open(fileURL) as f:
        for line in f:
            if re.search(r"%s"%satName, line):
                changeURL = re.sub('http://data.aodn.org.au', 'http://thredds.aodn.org.au/thredds/dodsC', line)
                getFiles.append(changeURL)
                
    return getFiles

In [4]:
jason2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-2')

jason3URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-3')

saralURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SARAL')

sentinel3aURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SENTINEL-3A')

cryosat2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'CRYOSAT-2')

enviURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ENVISAT')

geosatURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GEOSAT')

ersURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ERS-2')

gfoURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GFO')

topURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'TOPEX')

In [5]:
allURL = []
allURL.append(jason2URL)
allURL.append(jason3URL)
allURL.append(saralURL)
allURL.append(sentinel3aURL)
allURL.append(cryosat2URL)
allURL.append(enviURL)
allURL.append(geosatURL)
allURL.append(ersURL)
allURL.append(gfoURL)
allURL.append(topURL)

In [6]:
ncs = NetCDFFile(allURL[0][0])
lats = ncs.variables['LATITUDE'][:]
lons = ncs.variables['LONGITUDE'][:]

In [7]:
# time_var = allURL.variables['TIME']
# dtime = netCDF4.num2date(time_var[:],time_var.units)


In [8]:
# ncs = NetCDFFile(jason2URL[0])
# ncs.variables['TIME'][0]
# dtime2 = netCDF4.num2date(ncs.variables['TIME'][0:150],time_var.units)
# print dtime2

In [9]:
dtime2 = netCDF4.num2date(ncs[0:],units = "days since 1985-01-01 00:00:00 UTC", calendar='gregorian')
print dtime2

AttributeError: 'slice' object has no attribute 'startswith'

In [10]:
boxLat = []
boxLon = []
boxWh = []
boxTime = []

In [11]:
#Cyclone coordinates for marcia
latmin = -20
latmax = -15 
lonmin = 150
lonmax = 156

if latmin>latmax:
    print('Error wrong definition of min and max lat!!!')

if lonmin>lonmax:
    print('Error wrong definition of min and max lon!!!')

In [12]:
# Marcia dates
start_date = dt.datetime(2018,2,15)
end_date = dt.datetime(2018,2,21)


In [13]:
start_date.hour

0

In [14]:
import xarray as xr

ImportError: No module named xarray

In [ ]:
import datetime

attrs = {'units': 'hours since 2000-01-01'}
time_var = ncs.variables['TIME']

ds = xr.Dataset({'TIME': ('TIME', [0, 1, 2, 3], attrs)})

xr.decode_cf(ds)

ds1 = xr.open_dataset('allURL')

# Works:
dsloc1 = ds1.sel(TIME='2016-05-02 10:00', longitude=155, latitude=-20, method='nearest')


# Works:
tmp    = ds1.sel(TIME=slice('2016-05-02 10:00', '2016-05-02 12:00'))
dsloc2 = tmp.sel(longitude=4.9, latitude=51.2, method='nearest')

In [ ]:
import dateutil.parser

nc = netCDF4.Dataset('allURL', 'r')

# all_times variable includes the time:units attribute
all_times = nci.variables['time']

sdt = dateutil.parser.parse("2015-07-20T00:00:00")
edt = dateutil.parser.parse("2015-07-24T23:00:00")

st_idx = netCDF4.date2index(sdt, all_times)
et_idx = netCDF4.date2index(edt, all_times)

data = nc.variables['temperature'][st_idx:et_idx+1,:] #I want to read between 2015-07-20 00:00 to 2015-07-24 23:00

In [ ]:
for u in range(25) :#len(allURL)):
    urlON = allURL[u]
    for k in range(len(urlON)):
        ncs = NetCDFFile(urlON[k])
        lats = ncs.variables['LATITUDE'][:]
        lons = ncs.variables['LONGITUDE'][:]
        if u == 2:
            wh = ncs.variables['SWH_KA_CAL'][:]
        else:
            wh = ncs.variables['SWH_KU_CAL'][:]

        # Get desired time interval  
        time_var = ncs.variables['TIME']
        t = ncs.variables['TIME'][:]
        
        id_2 = netCDF4.date2index(end_date,time_var,select='nearest')
        diff3 = start_date-netCDF4.num2date(t[id_2],time_var.units)
        
        diff2 = end_date-netCDF4.num2date(t[id_2],time_var.units)
        #dh1 = (diff1.seconds + diff1.days * 24 * 3600)/3600.
        dh2 = (diff2.seconds + diff2.days * 24 * 3600)/3600.
        dh3 = (diff3.seconds + diff3.days * 24 * 3600)/3600.

        id_st = -1
        id_et = -1
        if dh3<0.:
            id_st = netCDF4.date2index(start_date,time_var,select='after')
            
        if dh2>-24.:
            id_et = netCDF4.date2index(end_date,time_var,select='before')
            
        if id_st<id_et:
            print id_st,id_et,netCDF4.num2date(t[id_st],time_var.units)
            print netCDF4.num2date(t[id_et],time_var.units)
            # Check if satellite points are in the box
            id1 = lats>latmin
            id2 = lats<latmax
            id3 = lons>lonmin
            id4 = lons<lonmax
            id1 = id1*1
            id2 = id2*1
            id3 = id3*1
            id4 = id4*1
            tot = id1+id2+id3+id4
            
            # If this is the case then the sum should be 4
            inside = np.where(tot==4)[0]
            idtime = np.arange(id_st,id_et+1)

            idin = np.intersect1d(inside, idtime)

            if len(idin)>0:
                boxLat.append(lats[idin])
                boxLon.append(lons[idin])
                boxWh.append(wh[idin])
                boxTime.append(t[idin])

In [ ]:
dtime2 = netCDF4.num2date(ncs[0:],units = "days since 1985-01-01 00:00:00 UTC", calendar='gregorian')


In [ ]:
for u in range(5) :#len(allURL)):
    urlON = allURL[u]
    for k in range(len(urlON)):
        ncs = NetCDFFile(urlON[k])
        lats = ncs.variables['LATITUDE'][:]
        lons = ncs.variables['LONGITUDE'][:]
        if u == 2:
            wh = ncs.variables['SWH_KA_CAL'][:]
        else:
            wh = ncs.variables['SWH_KU_CAL'][:]

        # Get desired time interval  
        time = ncs.variables['TIME']
        
        id_2 = netCDF4.date2index(end_date,time,time.units, select='nearest')
        diff3 = start_date-netCDF4.num2date(time[id_2],time.units)
        
        diff2 = end_date-netCDF4.num2date(time[id_2],time.units)
       # dh1 = (diff1.seconds + diff1.days * 24 * 3600)/3600.
        dh2 = (diff2.seconds + diff2.days * 24 * 3600)/3600.
        dh3 = (diff3.seconds + diff3.days * 24 * 3600)/3600.

        id_st = -1
        id_et = -1
        if dh3<0.:
            id_st = netCDF4.date2index(start_date,time,select='after')
            
        if dh2>-24.:
            id_et = netCDF4.date2index(end_date,time,select='before')
            
        if id_st<id_et:
            print id_st,id_et,netCDF4.num2date(time[id_st],time.units)
            print netCDF4.num2date(tt[id_et],tt.units)
            # Check if satellite points are in the box
            id1 = lats>latmin
            id2 = lats<latmax
            id3 = lons>lonmin
            id4 = lons<lonmax
            id1 = id1*1
            id2 = id2*1
            id3 = id3*1
            id4 = id4*1
            tot = id1+id2+id3+id4
            
            # If this is the case then the sum should be 4
            inside = np.where(tot==4)[0]
            idtime = np.arange(id_st,id_et+1)

            idin = np.intersect1d(inside, idtime)

            if len(idin)>0:
                boxLat.append(lats[idin])
                boxLon.append(lons[idin])
                boxWh.append(wh[idin])
                boxTime.append(time[idin])

In [ ]:
for k in range(len(boxLat)):
    if k == 0:
        lat = boxLat[k]
        lon = boxLon[k]
        wh = boxWh[k]
        tt = boxT[k]
    else:
        lat = np.append(lat,boxLat[k])
        lon = np.append(lon,boxLon[k])
        wh = np.append(wh,boxWh[k])
        tt = np.append(tt,boxT[k])

In [ ]:
print 'records for Olga in CBG Box 2:', len(boxWh)

In [ ]:
# print len(lat)
# print len(lon)
# print len(wh)
# print len(tt)

In [ ]:
#wave height pos values

ids = np.where(wh>0)[0]
whpos = wh[ids]
print len(whpos)

In [ ]:
# print np.mean(whpos)
# print max(whpos)
# print min (whpos)
# print  min(lon[ids])
# print min(lon)
# print max(lon)
# print  min(lat)
# print max(lat)
print min(netCDF4.num2date(boxT[0],time_var.units))
# print min (netCDF4.num2date(tt[ids],tt.units))
# print max (netCDF4.num2date(tt,tt.units))
# print max (netCDF4.num2date(tt[ids],tt.units))

In [ ]:
start_date.year

In [ ]:

# start_date = dt.datetime(2018,2,15)
# end_date = dt.datetime(2018,2,21)
time = netCDF4.num2date(tt[ids],tt.units)
dt = time-start_date
dt2 = end_date-time
dhours = -np.ones(len(dt))
dhours2 = -np.ones(len(dt2))


for k in range(len(dt)):
    if start_date.year == time[k].year:
        dhours[k] = (dt[k].seconds + dt[k].days * 24 * 3600)/3600.
        dhours2[k] = (dt2[k].seconds + dt2[k].days * 24 * 3600)/3600.

    

In [ ]:
time[700].year

In [ ]:
idds = np.where(np.logical_and(dhours>0.,dhours2>0.))[0]

In [ ]:
time2 = netCDF4.num2date(tt[idds],tt.units)
print 'all records for olga in CBG box 2:', time2[0:1000]

In [ ]:
np.sort(time2)[0:1000]

In [ ]:
time = netCDF4.num2date(tt[ids],tt.units)
print 'all records for olga in CBG box 2:', np.sort(time[0:1000])

In [ ]:
#find wh at a certain time period - 24 -  29th jan 2010 cyclone olga


time = netCDF4.num2date(tt[ids],tt.units)
print 'olga time records:', time[0:1000]


olga = netCDF4.date2index(start_date,tt,select='nearest')

In [ ]:
# cyclone Olga dates
start_olga = dt.datetime(2010,1,21)
end_olga = dt.datetime(2010,1,30)
s_olga = netCDF4.date2index(start_olga,tt,select='nearest')
e_olga = netCDF4.date2index(end_olga,tt,select='nearest')



# id_st = netCDF4.date2index(start_date,tt,select='nearest')
#         id_et = netCDF4.date2index(end_date,tt,select='nearest')

In [ ]:
print s_olga, e_olga

In [ ]:
olga = netCDF4.num2date(s_olga,tt.units)


In [ ]:
olga

In [ ]:
df.loc[df['time'].isin(['2015-2', '2015-03'])]